In [2]:
import torch_geometric.transforms as T


import pandas as pd
import json
import numpy as np
from functools import reduce
import torch
from torch_geometric.utils import remove_self_loops, to_undirected

from typing import Optional, Callable, List
import os.path as osp

import numpy as np
import torch
from torch_geometric.data import InMemoryDataset
from torch_geometric.data import Data
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from numpy import random
import sys
import scipy.io
import pandas as pd
import numpy as np
import sys
import math
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler


def generate_5CV_set(drivers,nondrivers,randseed):
    """
    Generate 5CV splits.
    :param drivers: List of canonical driver genes(positive samples)
    :param nondrivers: List of nondriver genes(negative samples)
    :param randseed: Random seed
    :return: 5CV splits sorted in a dictionary
    """
    # StratifiedKFold
    X, y = drivers + nondrivers, np.hstack(([1]*len(drivers), [0]*len(nondrivers)))
    skf = StratifiedKFold(n_splits=8,shuffle=True,random_state=randseed)
    X_5CV = {}
    cv_idx=1
    for train, test in skf.split(X, y):
        # train/test sorts the sample indices in X list.
        # For each split, we should convert the indices in train/test to names
        train_set=[]
        test_set=[]
        for i in train:
            train_set.append(X[i])
        for i in test:
            test_set.append(X[i])


        X_5CV['train_%d' % cv_idx] = torch.tensor(train_set)
        X_5CV['test_%d' % cv_idx] = torch.tensor(test_set)
        cv_idx += 1
    return X_5CV




delimiter = " "
net_name = "./data/string_net.txt"
#net_name = "/home/hwen6/database/string_db/string_v9.1.txt"
#net_name = "/home/hwen6/gongju/one_net_subnetwork/train_valid_G.csv"
#network_name = net_name.split("/")[-1].split("_")[0]
network_name = "string_version2"

# net_names = ["./data/string_net.txt",
#              "./data/BIOGRID_net.txt",
#              "./data/CPDB_net.txt",
#              "./data/HumanNet_net.txt",
#              "./data/IREF_net.txt",
#              "./data/PathwayCommons_net.txt",
#              "./data/pcnet_net.txt"]




graphs = [pd.read_csv(net_name, delimiter=delimiter, header=None)]
for G in graphs:
    if G.shape[1] < 3:
        G[2] = pd.Series([1.0] * len(G))

labels = []
#label_names = ["./data/mouse_gene_labels.json"]
label_names = ["./data/dep_gene_labels.json"]

for label_name in label_names:
    with open(label_name, "r") as f:
        labels.append(json.load(f))


node_sets = [np.union1d(G[0].values, G[1].values) for G in graphs]
union = reduce(np.union1d, node_sets)

#weights = torch.FloatTensor([1.0 for G in graphs])
weights = torch.FloatTensor([1.0])

masks = torch.FloatTensor([np.isin(union, nodes) for nodes in node_sets])
masks = torch.t(masks)

mapper = {name: idx for idx, name in enumerate(union)}

e_lst = pd.read_table(filepath_or_buffer='./data/dep_common_essential_genes', sep='\t', header=None, index_col=None,
                  names=['essential'])

#e_lst = pd.read_table(filepath_or_buffer='./data/Mouse_essential_genes', sep='\t', header=None, index_col=None,
#                  names=['essential'])

e_lst = e_lst['essential'].values.tolist()

# Nonessential genes (negative samples)
ne_lst = pd.read_table(filepath_or_buffer='./data/dep_non_essential_genes', sep='\t', header=None, index_col=None, names=['nonessential'])

#ne_lst = pd.read_table(filepath_or_buffer='./data/Mouse_viable_genes', sep='\t', header=None,
#               index_col=None, names=['nonessential'])
ne_lst = ne_lst['nonessential'].values.tolist()

e_idx = [mapper[i] for i in e_lst if i in mapper and i in labels[0]]
ne_idx = [mapper[i] for i in ne_lst if i in mapper  and i in labels[0]]


n = 1
k_sets_net = dict()
for k in np.arange(0,10): # Randomly generate 5CV splits for ten times
    k_sets_net[k] = []
    randseed = (k+1)%100+(k+1)*5
    cv = generate_5CV_set(e_idx,ne_idx,randseed)
    for cv_idx in np.arange(1,6):
        a = cv["train_%d" % cv_idx]
        b = cv["test_%d" % cv_idx]
        random.shuffle(a)
        random.shuffle(b)
        test_mask = b
        train_mask = a[:int(len(cv["train_1"])/10*9)]
        valid_mask = a[int(len(cv["train_1"])/10*9):]
        k_sets_net[k].append((train_mask, valid_mask, test_mask))
        n += 1


from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
        #gene_feature = pd.read_csv('/home/hwen6/public_data/TCGA/XENA_pancancer/tcga_RSEM_gene_fpkm_gname_pc150.csv', sep=',',index_col=0)
        #gene_feature = pd.read_csv('/home/hwen6/gongju/DeepHE/data/gene_dna_pep_network_feature.csv', sep=',',index_col=0)
        #gene_feature = pd.read_csv('/home/hwen6/database/gene_expression_prediction/cancer_full_expression_pc50.csv', sep=',',index_col=0)
#gene_feature = pd.read_csv('./data/cancer_full_expression.tsv', sep='\t')
#pca = PCA(n_components=50)

#pca_data = gene_feature.drop(['sample'], axis=1)

#pca_data = pd.DataFrame(pca_data,index=union).fillna(0)
#reduced_matrix = pca.fit_transform(pca_data)
gene_feature = pd.read_csv('/home/hwen6/database/gene_expression_prediction/cancer_full_expression_pc50.csv', sep=',',index_col=0)
#reduced_matrix = pd.DataFrame(gene_feature)
gene_df = pd.read_csv('/home/hwen6/database/gene_expression_prediction/cancer_full_expression.tsv', sep='\t')
#reduced_matrix.index = gene_feature['sample']

pyg_graphs = []
for i in range(10):
    for cv_run in range(5):
        train_mask, valid_mask, test_mask = k_sets_net[i][cv_run]
        train_valid_idx_list = set(train_mask.tolist() + valid_mask.tolist())
        train_valid_gnames = {gname for gname, idx in mapper.items() if idx in train_valid_idx_list}
        train_valid_G = G[G[0].isin(train_valid_gnames) & G[1].isin(train_valid_gnames)]

        
        e_lst = pd.read_table(filepath_or_buffer='./data/dep_common_essential_genes', sep='\t', header=None, index_col=None,
                  names=['essential'])

        #e_lst = pd.read_table(filepath_or_buffer='./data/Mouse_essential_genes', sep='\t', header=None, index_col=None,
        #                  names=['essential'])

        e_lst = e_lst['essential'].values.tolist()

        # Nonessential genes (negative samples)
        ne_lst = pd.read_table(filepath_or_buffer='./data/dep_non_essential_genes', sep='\t', header=None, index_col=None, names=['nonessential'])

        #ne_lst = pd.read_table(filepath_or_buffer='./data/Mouse_viable_genes', sep='\t', header=None,
        #               index_col=None, names=['nonessential'])
        ne_lst = ne_lst['nonessential'].values.tolist()

        e_idx = [mapper[i] for i in e_lst if i in mapper and i in labels[0]]
        ne_idx = [mapper[i] for i in ne_lst if i in mapper  and i in labels[0]]

        train_valid_gene_df = gene_df[gene_df['sample'].isin(train_valid_gnames)]

        
        pca = PCA(n_components=50)

        train_valid_pca_data = train_valid_gene_df.drop(['sample'], axis=1)

        all_data_f = gene_df[gene_df['sample'].isin(union)]

        all_data_f = pd.DataFrame(all_data_f,index=union).fillna(0)
        all_data = all_data_f.drop(['sample'], axis=1)


        #scaler = StandardScaler().fit(train_valid_pca_data)
        #train_scaled = scaler.transform(train_valid_pca_data)

        #all_scaled = scaler.transform(all_data)

        reduced_matrix = pca.fit(train_valid_pca_data)

        

        all_pca = pca.transform(all_data)

        scaler = preprocessing.MinMaxScaler()


        gene_feature_index = pd.DataFrame(all_pca).reindex(union, fill_value=0)
        feat_raw = scaler.fit_transform(np.abs(gene_feature_index))


        

        #gene_feature = pd.read_csv('/home/hwen6/gongju/DeepHE/data/gene_dna_pep_feature_gname_uniq.csv', sep=',',index_col=0)
        gene_feature_index = pd.DataFrame(gene_feature,index=union).fillna(0)

        from sklearn import preprocessing
        scaler = preprocessing.MinMaxScaler()
        feat_raw = scaler.fit_transform(np.abs(gene_feature_index))

        final_labels = []

In [3]:
feat_raw

array([[0.02116056, 0.08041215, 0.27146386, ..., 0.10360078, 0.08012268,
        0.1148902 ],
       [0.45162576, 0.12093439, 0.73567221, ..., 0.04582405, 0.08049637,
        0.14412393],
       [0.6770235 , 0.14258315, 0.20089181, ..., 0.06759901, 0.02039906,
        0.04266136],
       ...,
       [0.33420198, 0.08189742, 0.02921014, ..., 0.02446723, 0.02263919,
        0.02478441],
       [0.55236118, 0.04355629, 0.01502991, ..., 0.04594499, 0.09041962,
        0.04536995],
       [0.30603614, 0.00759395, 0.0081904 , ..., 0.02769054, 0.02851045,
        0.01100272]])

In [4]:

from utils.utils import prepare_folder
from utils.evaluator import Evaluator
from torch_geometric.data import NeighborSampler
from models import SAGE_NeighSampler
from tqdm import tqdm

import argparse

import torch
import torch.nn.functional as F
import torch.nn as nn
import pickle

import torch_geometric.transforms as T
from torch_sparse import SparseTensor
from torch_geometric.utils import to_undirected
import pandas as pd

import argparse

import torch
import torch.nn.functional as F
import torch.nn as nn

import torch_geometric.transforms as T
from torch_sparse import SparseTensor
from torch_geometric.utils import to_undirected
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
import pickle

eval_metric = 'auc'
import time

In [5]:
def load_obj( name ):
    """
    Load dataset from pickle file.
    :param name: Full pathname of the pickle file
    :return: Dataset type of dictionary
    """
    with open( name , 'rb') as f:
        return pickle.load(f)

In [65]:
dataset = load_obj("test_cancer_full_expression_pc50_PCA_exclude_test_string_version2.pkl")

In [67]:
data = dataset[0]

In [68]:
len(data.x)

13137

In [69]:
data.y

tensor([[-9223372036854775808],
        [-9223372036854775808],
        [                   0],
        ...,
        [                   0],
        [                   0],
        [                   1]])

In [70]:
data.adj_t = data.adj_t.to_symmetric()

In [71]:
split_idx = {'train':data.train_mask, 'valid':data.valid_mask, 'test':data.test_mask}
train_idx = split_idx['train']

In [72]:
len(train_idx)

8145

In [79]:
train_idx

tensor([   11,    11,    11,  ..., 10706,  9656,  5610])

In [73]:
len(data.valid_mask)

905

In [74]:
len(data.test_mask)

1293

In [77]:
data.test_mask

tensor([  49,   49,  116,  ..., 8778, 6702, 4939])

In [83]:
len(data.y)

13137

In [84]:
len(data.x)

13137

In [82]:
data.y[data.train_mask]

tensor([[1],
        [1],
        [1],
        ...,
        [0],
        [0],
        [0]])

In [ ]:


my_list = list(data.y[data.valid_mask])
unique_list = []
[unique_list.append(x) for x in my_list if x not in unique_list]
print(unique_list)


[tensor([1]), tensor([0])]


In [41]:
len(x)

13137

In [40]:
x = data.x
x = (x-x.mean(0))/x.std(0)
data.x = x
if data.y.dim()==2:
    data.y = data.y.squeeze(1)

In [85]:
from utils.utils import prepare_folder
from utils.evaluator import Evaluator
from torch_geometric.data import NeighborSampler
from models import SAGE_NeighSampler
from tqdm import tqdm

import argparse

import torch
import torch.nn.functional as F
import torch.nn as nn
import pickle

import torch_geometric.transforms as T
from torch_sparse import SparseTensor
from torch_geometric.utils import to_undirected
import pandas as pd

import argparse

import torch
import torch.nn.functional as F
import torch.nn as nn

import torch_geometric.transforms as T
from torch_sparse import SparseTensor
from torch_geometric.utils import to_undirected
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
import pickle

eval_metric = 'auc'
import time


sage_neighsampler_parameters = {'lr':0.003
              , 'num_layers':2
              , 'hidden_channels':128
              , 'dropout':0.0
              , 'batchnorm': False
              , 'l2':5e-7
             }



def train(epoch, train_loader, model, data, train_idx, optimizer, device, no_conv=False):
    model.train()

    pbar = tqdm(total=train_idx.size(0), ncols=80)
    pbar.set_description(f'Epoch {epoch:02d}')

    total_loss = total_correct = 0
    for batch_size, n_id, adjs in train_loader:
        # `adjs` holds a list of `(edge_index, e_id, size)` tuples.
        adjs = [adj.to(device) for adj in adjs]

        optimizer.zero_grad()
        out = model(data.x[n_id], adjs)
        loss = F.nll_loss(out, data.y[n_id[:batch_size]])
        loss.backward()
        optimizer.step()

        total_loss += float(loss)
        pbar.update(batch_size)

    pbar.close()
    loss = total_loss / len(train_loader)

    return loss


@torch.no_grad()
def test(layer_loader, model, data, split_idx, device, no_conv=False):
    # data.y is labels of shape (N, ) 
    model.eval()
    
    out = model.inference(data.x, layer_loader, device)
#     out = model.inference_all(data)
    y_pred = out.exp()  # (N,num_classes)   
    
    losses = dict()
    for key in ['train', 'valid', 'test']:
        node_id = split_idx[key]
        node_id = node_id.to(device)
        losses[key] = F.nll_loss(out[node_id], data.y[node_id]).item()
            
    return losses, y_pred

@torch.no_grad()
def inference_test(layer_loader, model, data, device, no_conv=False):
    # data.y is labels of shape (N, ) 
    model.eval()
    
    out = model.inference(data.x, layer_loader, device)
#     out = model.inference_all(data)
    y_pred = out.exp()  # (N,num_classes)   
                
    return y_pred

def load_obj( name ):
    """
    Load dataset from pickle file.
    :param name: Full pathname of the pickle file
    :return: Dataset type of dictionary
    """
    with open( name , 'rb') as f:
        return pickle.load(f)

In [87]:
dataset = load_obj("test_cancer_full_expression_pc50_PCA_exclude_test_string_version2.pkl")

device = f'cuda:{0}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

In [ ]:
nlabels = 2
    #networks_name = ['BioGrid','CPDB','pcnet','pcnet','pcnet','pcnet','string']
  
for n in range(1):
    data = dataset[n]
    data.adj_t = data.adj_t.to_symmetric()


    model_dir = prepare_folder("dep_pc50_version2_{}_{}".format('string', n), 'sage_neighsampler')
    split_idx = {'train':data.train_mask, 'valid':data.valid_mask, 'test':data.test_mask}
    train_idx = split_idx['train'].to(device)
    data = data.to(device)
        

    x = data.x
    x = (x-x.mean(0))/x.std(0)
    data.x = x
    if data.y.dim()==2:
        data.y = data.y.squeeze(1)

    data = data.to(device)

    train_loader = NeighborSampler(data.adj_t, node_idx=train_idx, sizes=[25, 10], batch_size=1024, shuffle=True, num_workers=12)

    layer_loader = NeighborSampler(data.adj_t, node_idx=None, sizes=[-1], batch_size=4096, shuffle=False, num_workers=12)    

    para_dict = sage_neighsampler_parameters
    model_para = sage_neighsampler_parameters.copy()
    model_para.pop('lr')
    model_para.pop('l2')
    model = SAGE_NeighSampler(in_channels = data.x.size(-1), out_channels = nlabels, **model_para).to(device)

    model.reset_parameters()
    optimizer = torch.optim.Adam(model.parameters(), lr=para_dict['lr'], weight_decay=para_dict['l2'])
    min_valid_loss = 1e8

    
    

In [90]:
layer_loader

NeighborSampler(sizes=[-1])